# Working with NV Aggregated Data

Pre-processing input data for a smoother upload experience of the state data to the WaDE 2.0 database.
Using geopandas to read in shp file, and coverting to WKT for ReportingUnit geometry.

#### Notes:
- NA

In [1]:
#Needed Libararies
import os
import numpy as np
import pandas as pd
from datetime import datetime
import geopandas as gpd # the library that lets us read in shapefiles
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

#Setting work directory, reading inputs, creating dataframe
workingDir = "G:/Shared drives/WaDE Data/Nevada/AggregatedAmounts/RawInputData"
os.chdir(workingDir)

# Pumpage Data

In [2]:
#CSV input file
basin2015Input = "StateInv_2015_BasinPumpage.csv"
basin2017Input = "StateInv_2017_BasinPumpage.csv"
county2015Input = "StateInv_2015_CountyPumpage.csv"
county2017Input = "StateInv_2015_CountyPumpage.csv"

In [3]:
# basin 2015
dfb15 = pd.read_csv(basin2015Input)
dfb15['Year'] = "2015"
dfb15['in_ReportingUnitType'] = "Basin"
print(len(dfb15))
dfb15.head(1)

256


,OID_,BasinID,BasinName,Total,COM,CON,DOM,ENV,IND,IRR,MM,MUN,OTH,PWR,QM,REC,STK,WLD,Shape_Length,Shape_Area,Year,in_ReportingUnitType
0,1,009,Long Valley,1245.0,0.0,0.0,6.0,0.0,0.0,1103.0,0.0,0.0,0.0,0,0.0,0.0,136,0.0,225645.674761,1.126936e+09,2015,Basin


In [4]:
# basin 2017
dfb17 = pd.read_csv(basin2017Input)
dfb17['Year'] = "2017"
dfb17['in_ReportingUnitType'] = "Basin"
print(len(dfb17))
dfb17.head(1)

256


,OID_,BasinID,BasinName,Total,COM,CON,DOM,ENV,IND,IRR,MM,MUN,OTH,PWR,QM,REC,STK,WLD,GlobalID,Shape_Length,Shape_Area,Year,in_ReportingUnitType
0,1,009,Long Valley,297.0,0.0,0.0,6.0,0.0,0.0,155.0,0.0,0.0,0.0,0,0.0,0.0,136,0.0,{00000000-0000-0000-0000-000000000000},225645.674761,1.126936e+09,2017,Basin


In [5]:
# coutny 2015
dfc15 = pd.read_csv(county2015Input)
dfc15['Year'] = "2015"
dfc15['in_ReportingUnitType'] = "County"
print(len(dfc15))
dfc15.head(1)

17


,OID_,County,Total,COM,CON,DOM,ENV,IND,IRR,MM,MUN,OTH,PWR,QM,REC,STK,WLD,GlobalID,Shape_Length,Shape_Area,Year,in_ReportingUnitType
0,1,Clark,98420.244,1363.22,70.58,5779.5,2784.16,4125.42,5216.0,2197.3,59698.16,977.0,0,10663.454,5410.45,21,114.0,{00000000-0000-0000-0000-000000000000},734866.608384,2.087734e+10,2015,County


In [6]:
# coutny 2017
dfc17 = pd.read_csv(county2017Input)
dfc17['Year'] = "2017"
dfc17['in_ReportingUnitType'] = "County"
print(len(dfc17))
dfc17.head(1)

17


,OID_,County,Total,COM,CON,DOM,ENV,IND,IRR,MM,MUN,OTH,PWR,QM,REC,STK,WLD,GlobalID,Shape_Length,Shape_Area,Year,in_ReportingUnitType
0,1,Clark,98420.244,1363.22,70.58,5779.5,2784.16,4125.42,5216.0,2197.3,59698.16,977.0,0,10663.454,5410.45,21,114.0,{00000000-0000-0000-0000-000000000000},734866.608384,2.087734e+10,2017,County


### Concatenate Files

In [7]:
# Concatenate basin dataframes.
frames = [dfb15, dfb17]
dfbtemp = pd.concat(frames).reset_index(drop=True)
dfbtemp['in_ReportingUnitNativeID'] = dfbtemp['BasinID']
dfbtemp['in_ReportingUnitName'] = dfbtemp['BasinName']
dfbtemp = dfbtemp.drop(['OID_', 'BasinID', 'BasinName'], axis=1)
print(len(dfbtemp))
dfbtemp.head(3)

512


,Total,COM,CON,DOM,ENV,IND,IRR,MM,MUN,OTH,PWR,QM,REC,STK,WLD,Shape_Length,Shape_Area,Year,in_ReportingUnitType,GlobalID,in_ReportingUnitNativeID,in_ReportingUnitName
0,1245.00,0.0,0.0,6.0,0.0,0.0,1103.0,0.0,0.00,0.0,0,0.0,0.0,136,0.0,225645.674761,1.126936e+09,2015,Basin,NaN,009,Long Valley
1,3292.03,0.0,0.0,42.0,0.0,0.0,3124.0,0.0,59.33,0.0,0,46.7,0.0,20,0.0,233873.918113,1.516127e+09,2015,Basin,NaN,033B,Quinn River Valley
2,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0,0.0,0.0,0,0.0,47416.631321,8.575338e+07,2015,Basin,NaN,012,Mosquito Valley


In [8]:
# need to transpose the data.
# create Temporary basin out dataframe
columnsList = ["Year", "in_ReportingUnitType", "in_ReportingUnitNativeID", "in_ReportingUnitName"]
dfbtemp2 = pd.DataFrame(columns=columnsList)
dfbtemp2 = dfbtemp[columnsList]

############################################

dfbtemp2 = dfbtemp2.assign(NV_BenUse='')
dfbtemp2 = dfbtemp2.assign(in_Amount='')
dfbtemp2 = dfbtemp2.assign(in_ReportingUnitType='')
dfBasinOut = pd.DataFrame() # dataframe to append to

############################################
columnsList = [
    'COM',
    'CON',
    'DOM',
    'ENV',
    'IND',
    'IRR',
    'MM',
    'MUN',
    'OTH',
    'PWR',
    'QM',
    'REC',
    'STK',
    'WLD']
lenList = len(columnsList)


############################################
for i in range(lenList):
    BenuseString = columnsList[i]
    dfbtemp2['NV_BenUse'] = BenuseString
    dfbtemp2['in_Amount'] = dfbtemp[columnsList[i]]
    dfbtemp2['in_ReportingUnitType'] = "Basin"
    dfBasinOut = dfBasinOut.append(dfbtemp2)
    
############################################

print(len(dfBasinOut.index))
dfBasinOut

7168


C:\Users\rjame\AppData\Local\Temp\ipykernel_22176\896595979.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfBasinOut = dfBasinOut.append(dfbtemp2)
C:\Users\rjame\AppData\Local\Temp\ipykernel_22176\896595979.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfBasinOut = dfBasinOut.append(dfbtemp2)
C:\Users\rjame\AppData\Local\Temp\ipykernel_22176\896595979.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfBasinOut = dfBasinOut.append(dfbtemp2)
C:\Users\rjame\AppData\Local\Temp\ipykernel_22176\896595979.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfBasinOut = dfBasinOut.append(dfbtemp2)
C:\Users\rjame\AppData\Local

,Year,in_ReportingUnitType,in_ReportingUnitNativeID,in_ReportingUnitName,NV_BenUse,in_Amount
0,2015,Basin,009,Long Valley,COM,0.00
1,2015,Basin,033B,Quinn River Valley,COM,0.00
2,2015,Basin,012,Mosquito Valley,COM,0.00
3,2015,Basin,007,Swan Lake Valley,COM,0.00
4,2015,Basin,029,Pine Forest Valley,COM,4.48
...,...,...,...,...,...,...
507,2017,Basin,218,California Wash,WLD,0.00
508,2017,Basin,223,Gold Butte Area,WLD,0.00
509,2017,Basin,215,Black Mountains Area,WLD,0.00
510,2017,Basin,107,Smith Valley,WLD,0.00


In [9]:
# Concatenate county dataframes.
frames = [dfc15, dfc17]
dfctemp = pd.concat(frames).reset_index(drop=True)
dfctemp['in_ReportingUnitNativeID'] = ""
dfctemp['in_ReportingUnitName'] = dfctemp['County']
dfctemp = dfctemp.drop(['OID_', 'County'], axis=1)
print(len(dfctemp))
dfctemp.head(3)

34


,Total,COM,CON,DOM,ENV,IND,IRR,MM,MUN,OTH,PWR,QM,REC,STK,WLD,GlobalID,Shape_Length,Shape_Area,Year,in_ReportingUnitType,in_ReportingUnitNativeID,in_ReportingUnitName
0,98420.244000,1363.220000,70.58,5779.5,2784.16000,4125.420000,5216.00,2197.300000,59698.160000,977.0,0,10663.454000,5410.45,21,114.00,{00000000-0000-0000-0000-000000000000},734866.608384,2.087734e+10,2015,County,,Clark
1,12398.606126,34.929285,0.00,85.0,8.72708,72.303284,6544.00,2457.875809,944.925567,0.0,0,1882.355101,0.00,64,304.49,{00000000-0000-0000-0000-000000000000},483551.122120,9.870862e+09,2015,County,,Mineral
2,113282.636403,2.000000,0.00,584.0,2217.00000,206.430000,99417.45,7695.166403,1111.000000,0.0,1177,349.590000,248.00,275,0.00,{00000000-0000-0000-0000-000000000000},632704.546310,1.428683e+10,2015,County,,Lander


In [10]:
# Creating WaDE Custom reportingunit native ID for easy site identificaiion
# ----------------------------------------------------------------------------------------------------

# Create temp ReportingUnitNativeID dataframe of unique reporting units.
def assignReportingUnitNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "WaDENV_RU" + string1
    return outstring

dfReportingUnitNativeID = pd.DataFrame()
dfReportingUnitNativeID['in_ReportingUnitName'] = dfctemp['in_ReportingUnitName']
dfReportingUnitNativeID['in_ReportingUnitType'] = dfctemp['in_ReportingUnitType']
dfReportingUnitNativeID = dfReportingUnitNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfReportingUnitNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfReportingUnitNativeID['in_ReportingUnitNativeID'] = dftemp.apply(lambda row: assignReportingUnitNativeID(row['Count']), axis=1)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom reportingunit native ID
def retrieveReportingUnitNativeID(A, B):
    if (A == '' and B == '') or (pd.isnull(A) and pd.isnull(B)):
        outList = ''
    else:
        ml = dfReportingUnitNativeID.loc[(dfReportingUnitNativeID['in_ReportingUnitName'] == A) & 
                                         (dfReportingUnitNativeID['in_ReportingUnitType'] == B), 'in_ReportingUnitNativeID']
        if not (ml.empty):  # check if the series is empty
            outList = ml.iloc[0]
        else:
            outList = ''
    return outList

dfctemp['in_ReportingUnitNativeID'] = dfctemp.apply(lambda row: retrieveReportingUnitNativeID( row['in_ReportingUnitName'], row['in_ReportingUnitType']), axis=1)
dfctemp

,Total,COM,CON,DOM,ENV,IND,IRR,MM,MUN,OTH,PWR,QM,REC,STK,WLD,GlobalID,Shape_Length,Shape_Area,Year,in_ReportingUnitType,in_ReportingUnitNativeID,in_ReportingUnitName
0,98420.244000,1363.220000,70.580000,5779.5,2784.160000,4125.420000,5216.00,2197.300000,59698.160000,977.00,0,10663.454000,5410.450000,21,114.000,{00000000-0000-0000-0000-000000000000},7.348666e+05,2.087734e+10,2015,County,WaDENV_RU1,Clark
1,12398.606126,34.929285,0.000000,85.0,8.727080,72.303284,6544.00,2457.875809,944.925567,0.00,0,1882.355101,0.000000,64,304.490,{00000000-0000-0000-0000-000000000000},4.835511e+05,9.870862e+09,2015,County,WaDENV_RU2,Mineral
2,113282.636403,2.000000,0.000000,584.0,2217.000000,206.430000,99417.45,7695.166403,1111.000000,0.00,1177,349.590000,248.000000,275,0.000,{00000000-0000-0000-0000-000000000000},6.327045e+05,1.428683e+10,2015,County,WaDENV_RU3,Lander
3,30614.870671,767.920000,1.252111,4145.0,151.160169,12138.666133,6691.00,8.560000,3046.358908,0.00,0,2867.217350,130.720000,341,326.016,{00000000-0000-0000-0000-000000000000},5.566460e+05,1.300982e+10,2015,County,WaDENV_RU4,Churchill
4,119487.620000,14.000000,0.000000,215.0,17.000000,1668.860000,99173.50,15655.090000,1704.000000,0.00,0,543.170000,0.000000,497,0.000,{00000000-0000-0000-0000-000000000000},5.383747e+05,1.081974e+10,2015,County,WaDENV_RU5,Eureka
5,69876.317592,1425.726041,33.350000,8984.0,4489.390000,2864.450000,18217.83,31.000000,21238.751604,80.43,0,8644.208138,1580.131808,1076,1211.050,{00000000-0000-0000-0000-000000000000},7.885534e+05,1.694552e+10,2015,County,WaDENV_RU6,Washoe
6,145085.252313,1630.370000,1.000000,4581.0,0.140000,4596.340000,115612.00,265.170000,7260.492313,0.00,0,1522.740000,5990.000000,254,3372.000,{00000000-0000-0000-0000-000000000000},4.734803e+05,5.244387e+09,2015,County,WaDENV_RU7,Lyon
7,41956.405321,205.960000,0.000000,223.0,0.000000,0.000000,28954.00,12120.830000,46.076398,1.00,0,158.538923,167.000000,80,0.000,{00000000-0000-0000-0000-000000000000},4.303518e+05,9.287198e+09,2015,County,WaDENV_RU8,Esmeralda
8,6515.367000,25.827000,0.000000,1032.0,1.390000,2.950000,18.30,0.000000,5350.420000,0.00,0,82.480000,0.000000,2,0.000,{00000000-0000-0000-0000-000000000000},1.128603e+05,4.075041e+08,2015,County,WaDENV_RU9,Carson City
9,347228.304156,764.823226,0.028175,1841.0,241.390000,6234.980991,322829.00,10088.996000,3258.843689,183.62,0,652.412346,838.179730,295,0.030,{00000000-0000-0000-0000-000000000000},7.133020e+05,2.500068e+10,2015,County,WaDENV_RU10,Humboldt


In [11]:
# need to transpose the data.
# create Temporary county out dataframe
columnsList = ["Year", "in_ReportingUnitType", "in_ReportingUnitNativeID", "in_ReportingUnitName"]
dfctemp2 = pd.DataFrame(columns=columnsList)
dfctemp2 = dfctemp[columnsList]

############################################

dfctemp2 = dfctemp2.assign(NV_BenUse='')
dfctemp2 = dfctemp2.assign(in_Amount='')
dfctemp2 = dfctemp2.assign(in_ReportingUnitType='')
dfCountyOut = pd.DataFrame() # dataframe to append to

############################################
columnsList = [
    'COM',
    'CON',
    'DOM',
    'ENV',
    'IND',
    'IRR',
    'MM',
    'MUN',
    'OTH',
    'PWR',
    'QM',
    'REC',
    'STK',
    'WLD']
lenList = len(columnsList)


############################################
for i in range(lenList):
    BenuseString = columnsList[i]
    dfctemp2['NV_BenUse'] = BenuseString
    dfctemp2['in_Amount'] = dfctemp[columnsList[i]]
    dfctemp2['in_ReportingUnitType'] = "County"
    dfCountyOut = dfCountyOut.append(dfctemp2)
    
############################################

print(len(dfCountyOut.index))
dfCountyOut

476


C:\Users\rjame\AppData\Local\Temp\ipykernel_22176\1428374340.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfCountyOut = dfCountyOut.append(dfctemp2)
C:\Users\rjame\AppData\Local\Temp\ipykernel_22176\1428374340.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfCountyOut = dfCountyOut.append(dfctemp2)
C:\Users\rjame\AppData\Local\Temp\ipykernel_22176\1428374340.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfCountyOut = dfCountyOut.append(dfctemp2)
C:\Users\rjame\AppData\Local\Temp\ipykernel_22176\1428374340.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfCountyOut = dfCountyOut.append(dfctemp2)
C:\Users\rjame\A

,Year,in_ReportingUnitType,in_ReportingUnitNativeID,in_ReportingUnitName,NV_BenUse,in_Amount
0,2015,County,WaDENV_RU1,Clark,COM,1363.220000
1,2015,County,WaDENV_RU2,Mineral,COM,34.929285
2,2015,County,WaDENV_RU3,Lander,COM,2.000000
3,2015,County,WaDENV_RU4,Churchill,COM,767.920000
4,2015,County,WaDENV_RU5,Eureka,COM,14.000000
...,...,...,...,...,...,...
29,2017,County,WaDENV_RU13,Lincoln,WLD,59.980000
30,2017,County,WaDENV_RU14,Nye,WLD,5.000000
31,2017,County,WaDENV_RU15,Pershing,WLD,6.970000
32,2017,County,WaDENV_RU16,Storey,WLD,1.000000


In [12]:
# Concatenate temp basin and county dataframes.
frames = [dfBasinOut, dfCountyOut]
dfout = pd.concat(frames).reset_index(drop=True)
print(len(dfout))
dfout

7644


,Year,in_ReportingUnitType,in_ReportingUnitNativeID,in_ReportingUnitName,NV_BenUse,in_Amount
0,2015,Basin,009,Long Valley,COM,0.00
1,2015,Basin,033B,Quinn River Valley,COM,0.00
2,2015,Basin,012,Mosquito Valley,COM,0.00
3,2015,Basin,007,Swan Lake Valley,COM,0.00
4,2015,Basin,029,Pine Forest Valley,COM,4.48
...,...,...,...,...,...,...
7639,2017,County,WaDENV_RU13,Lincoln,WLD,59.98
7640,2017,County,WaDENV_RU14,Nye,WLD,5.00
7641,2017,County,WaDENV_RU15,Pershing,WLD,6.97
7642,2017,County,WaDENV_RU16,Storey,WLD,1.00


In [13]:
# TimeframeStart & TimeframeEnd

dfout['in_TimeframeStart'] = '01/01/' + dfout['Year'].astype(str)
dfout['in_TimeframeEnd'] = '12/31/' + dfout['Year'].astype(str)
dfout

,Year,in_ReportingUnitType,in_ReportingUnitNativeID,in_ReportingUnitName,NV_BenUse,in_Amount,in_TimeframeStart,in_TimeframeEnd
0,2015,Basin,009,Long Valley,COM,0.00,01/01/2015,12/31/2015
1,2015,Basin,033B,Quinn River Valley,COM,0.00,01/01/2015,12/31/2015
2,2015,Basin,012,Mosquito Valley,COM,0.00,01/01/2015,12/31/2015
3,2015,Basin,007,Swan Lake Valley,COM,0.00,01/01/2015,12/31/2015
4,2015,Basin,029,Pine Forest Valley,COM,4.48,01/01/2015,12/31/2015
...,...,...,...,...,...,...,...,...
7639,2017,County,WaDENV_RU13,Lincoln,WLD,59.98,01/01/2017,12/31/2017
7640,2017,County,WaDENV_RU14,Nye,WLD,5.00,01/01/2017,12/31/2017
7641,2017,County,WaDENV_RU15,Pershing,WLD,6.97,01/01/2017,12/31/2017
7642,2017,County,WaDENV_RU16,Storey,WLD,1.00,01/01/2017,12/31/2017


In [14]:
# For creating BeneficialUse
BeneficialUseDict = {
    "COM" : "Commercial",
    "CON" : "Construction",
    "DOM" : "Domestic",
    "ENV" : "Environmental",
    "IND" : "Industrial",
    "IRR" : "Irrigation",
    "MM" : "Mining and Milling",
    "MUN" : "Municipal",
    "PWR" : "Power",
    "QM" : "Quasi-Municipal",
    "REC" : "Recreational",
    "STK" : "Stockwater",
    "WLD" : "Wildlife"}
def assignBeneficialUse(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = "Unspecified"
    else:
        String1 = colrowValue.strip()
        try:
            outList = BeneficialUseDict[String1]
        except:
            outList = "Unspecified"

    return outList

dfout['in_BeneficialUseCategory'] = dfout.apply(lambda row: assignBeneficialUse(row['NV_BenUse']), axis=1)
dfout['in_BeneficialUseCategory'].unique()

array(['Commercial', 'Construction', 'Domestic', 'Environmental',
       'Industrial', 'Irrigation', 'Mining and Milling', 'Municipal',
       'Unspecified', 'Power', 'Quasi-Municipal', 'Recreational',
       'Stockwater', 'Wildlife'], dtype=object)

In [15]:
# in_VariableSpecificCV Info
dfout['in_VariableSpecificCV'] = "Withdrawal_Annual_" + dfout['in_BeneficialUseCategory'] + "_Groundwater"
dfout['in_VariableSpecificCV'].unique()

array(['Withdrawal_Annual_Commercial_Groundwater',
       'Withdrawal_Annual_Construction_Groundwater',
       'Withdrawal_Annual_Domestic_Groundwater',
       'Withdrawal_Annual_Environmental_Groundwater',
       'Withdrawal_Annual_Industrial_Groundwater',
       'Withdrawal_Annual_Irrigation_Groundwater',
       'Withdrawal_Annual_Mining and Milling_Groundwater',
       'Withdrawal_Annual_Municipal_Groundwater',
       'Withdrawal_Annual_Unspecified_Groundwater',
       'Withdrawal_Annual_Power_Groundwater',
       'Withdrawal_Annual_Quasi-Municipal_Groundwater',
       'Withdrawal_Annual_Recreational_Groundwater',
       'Withdrawal_Annual_Stockwater_Groundwater',
       'Withdrawal_Annual_Wildlife_Groundwater'], dtype=object)

# Shapefile Data

In [16]:
# Basin Shapefile input
basinShape = gpd.read_file('shapefile/NVBasinShapefile.shp')
print(len(basinShape))
basinShape.head(3)

256


,BasinID,BasinName,Total,COM,CON,DOM,ENV,IND,IRR,MM,MUN,OTH,PWR,QM,REC,STK,WLD,GlobalID,Shape_Leng,Shape_Area,geometry
0,009,Long Valley,297.00,0.0,0.0,6.0,0.0,0.0,155.0,0.0,0.00,0.0,0,0.0,0.0,136,0.0,{D8338F68-4281-4E70-A4F9-F2B3CC075F52},225645.674761,1.126936e+09,"POLYGON ((275769.417 4649545.137, 275801.697 4..."
1,033B,Quinn River Valley,3316.77,0.0,0.0,40.0,0.0,0.0,3148.0,0.0,62.07,0.0,0,46.7,0.0,20,0.0,{86BC11B8-17FE-45EB-8B41-BF555A8B4972},233873.918113,1.516127e+09,"POLYGON ((409038.817 4650081.777, 409055.317 4..."
2,012,Mosquito Valley,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0,0.0,0.0,0,0.0,{6530C8D7-C690-4B0F-B5E2-D422EA50F82F},47416.631321,8.575338e+07,"POLYGON ((272861.427 4639227.637, 272832.477 4..."


In [17]:
columnsList = ['RU_Name', 'RU_Type', 'geometry']
dfNVBshape = pd.DataFrame(columns=columnsList)
dfNVBshape['RU_Name'] = basinShape['BasinName']
dfNVBshape['RU_Type'] = "Basin"
dfNVBshape['geometry'] = basinShape['geometry']
dfNVBshape = dfNVBshape.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
dfNVBshape.head(3)

,RU_Name,RU_Type,geometry
0,Long Valley,Basin,"POLYGON ((275769.417 4649545.137, 275801.697 4..."
1,Quinn River Valley,Basin,"POLYGON ((409038.817 4650081.777, 409055.317 4..."
2,Mosquito Valley,Basin,"POLYGON ((272861.427 4639227.637, 272832.477 4..."


In [18]:
# Coutny Shapefile input
countyShape = gpd.read_file('shapefile/NVCountyShapefile.shp')
print(len(countyShape))
countyShape.head(3)

17


,County,Total,COM,CON,DOM,ENV,IND,IRR,MM,MUN,OTH,PWR,QM,REC,STK,WLD,GlobalID,Shape_Leng,Shape_Area,geometry
0,Clark,96196.890800,1297.00,438.7608,5746.0,3240.160000,3238.250000,6209.000000,2989.160000,57176.770000,0.0,0,10203.340000,5494.45000,21,143.00,{083692DA-2B8B-47BD-8860-0E2BE4C35B1D},734866.608384,2.087734e+10,"POLYGON ((598351.594 4077922.300, 603409.938 4..."
1,Mineral,10993.603916,35.38,0.0000,96.0,6.510000,72.303284,6320.000000,1127.885809,1075.750000,0.0,0,1891.284823,0.00000,64,304.49,{F639C53D-05BE-4413-913F-D27C1A46EC25},483551.122120,9.870862e+09,"POLYGON ((311683.945 4254035.912, 314389.148 4..."
2,Lander,93094.801599,2.02,0.0000,601.0,1889.342258,996.637119,79413.285866,7554.372000,923.245557,0.0,1177,213.476469,49.42233,275,0.00,{63B78501-1500-4A7B-A513-9F3EE92B84CD},632704.546310,1.428683e+10,"POLYGON ((430281.696 4333219.088, 430535.828 4..."


In [19]:
columnsList = ['RU_Name', 'RU_Type', 'geometry']
dfNVCshape = pd.DataFrame(columns=columnsList)
dfNVCshape['RU_Name'] = countyShape['County']
dfNVCshape['RU_Type'] = "County"
dfNVCshape['geometry'] = countyShape['geometry']
dfNVCshape = dfNVCshape.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
dfNVCshape.head(3)

,RU_Name,RU_Type,geometry
0,Clark,County,"POLYGON ((598351.594 4077922.300, 603409.938 4..."
1,Mineral,County,"POLYGON ((311683.945 4254035.912, 314389.148 4..."
2,Lander,County,"POLYGON ((430281.696 4333219.088, 430535.828 4..."


In [20]:
# Concatenate basin and county shape dataframes together.
frames = [dfNVBshape, dfNVCshape]
dfAllShape = pd.concat(frames).reset_index(drop=True)
dfAllShape

,RU_Name,RU_Type,geometry
0,Long Valley,Basin,"POLYGON ((275769.417 4649545.137, 275801.697 4..."
1,Quinn River Valley,Basin,"POLYGON ((409038.817 4650081.777, 409055.317 4..."
2,Mosquito Valley,Basin,"POLYGON ((272861.427 4639227.637, 272832.477 4..."
3,Swan Lake Valley,Basin,"POLYGON ((306578.287 4638189.407, 306923.487 4..."
4,Pine Forest Valley,Basin,"POLYGON ((383016.817 4637824.387, 383964.507 4..."
...,...,...,...
268,Lincoln,County,"POLYGON ((598546.178 4080102.742, 598221.345 4..."
269,Pershing,County,"POLYGON ((300819.554 4447871.847, 301312.154 4..."
270,Storey,County,"POLYGON ((265826.091 4349049.498, 266933.188 4..."
271,Douglas,County,"POLYGON ((240318.631 4333577.538, 244258.944 4..."


### Inspect Output Data & Export

In [21]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(dfout.dtypes)

Year                         object
in_ReportingUnitType         object
in_ReportingUnitNativeID     object
in_ReportingUnitName         object
NV_BenUse                    object
in_Amount                   float64
in_TimeframeStart            object
in_TimeframeEnd              object
in_BeneficialUseCategory     object
in_VariableSpecificCV        object
dtype: object


In [22]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(dfAllShape.dtypes)

RU_Name       object
RU_Type       object
geometry    geometry
dtype: object


In [23]:
# Export out to CSV.
dfout.to_csv('P_nvAggMaster.csv', index=False) # The output.
dfAllShape.to_csv('P_nvGeometry.csv', index=False) # The output geometry.